In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim

import requests 
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

In [3]:
with open('newyork_data.json') as json_data:
    newyork = json.load(json_data)

In [4]:
NYdata = newyork['features']

In [5]:
ColumnName = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

neighborhoods = pd.DataFrame(columns=ColumnName)

In [6]:
for data in NYdata:
    borough = name = data['properties']['borough'] 
    name = data['properties']['name']
        
    latlon = data['geometry']['coordinates']
    lat = latlon[1]
    lon = latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': name,
                                          'Latitude': lat,
                                          'Longitude': lon}, ignore_index=True)

In [7]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [8]:
bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
bronx_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [9]:
address = 'Bronx, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [10]:
bronx_map = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(bronx_data['Latitude'], bronx_data['Longitude'], bronx_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(bronx_map) 
 
bronx_map

In [11]:
CLIENT_ID = 'FXKFOYJL4DUHJTBMFTTN3L24MFMRHUUTABFWUFXA03CLPOLM'
CLIENT_SECRET = 'RY0P4TOPIR3GXLNYRVP1KKRHUCJEBHZBHQGD5XMNDSCU2GDU'
VERSION = '20190704'

In [12]:
bronx_data.loc[15, 'Neighborhood']

'Fordham'

In [13]:
fordham_latitude = bronx_data.loc[15, 'Latitude']
fordham_longitude = bronx_data.loc[15, 'Longitude'] 

fordham_name = bronx_data.loc[15, 'Neighborhood'] 

In [14]:
LIMIT = 100
radius = 3000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    fordham_latitude, 
    fordham_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=FXKFOYJL4DUHJTBMFTTN3L24MFMRHUUTABFWUFXA03CLPOLM&client_secret=RY0P4TOPIR3GXLNYRVP1KKRHUCJEBHZBHQGD5XMNDSCU2GDU&v=20190704&ll=40.86099679638654,-73.89642655981623&radius=3000&limit=100'

In [15]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d1dabc4342adf00384a9e3a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bronx',
  'headerFullLocation': 'Bronx',
  'headerLocationGranularity': 'city',
  'totalResults': 234,
  'suggestedBounds': {'ne': {'lat': 40.887996823386565,
    'lng': -73.86079301420541},
   'sw': {'lat': 40.83399676938651, 'lng': -73.93206010542706}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5647571c498e97b3fdee10fd',
       'name': 'Fresh Frutii',
       'location': {'address': '2449 Creston Ave',
        'lat': 40.86172173610795,
        'lng': -73.89868241533442,
        'labeledLatLngs': [{'label': 'display',
       

In [16]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Fresh Frutii,Juice Bar,40.861722,-73.898682
1,Edgar Allan Poe Cottage,History Museum,40.865573,-73.894421
2,Corky's,Diner,40.863384,-73.896515
3,Primavera Cafe,Café,40.862004,-73.891537
4,Tino's Delicatessen,Italian Restaurant,40.855882,-73.887166


In [18]:
nearby_venues.loc[nearby_venues['categories'] == 'Zoo']


,name,categories,lat,lng
64,Bronx Zoo,Zoo,40.853107,-73.878094
65,Sea Lion Pool,Zoo,40.852882,-73.878545
69,Children's Zoo,Zoo,40.851826,-73.880127
71,Congo Gorilla Forest,Zoo,40.847774,-73.881604
98,Madagascar,Zoo,40.852627,-73.879281
